In [ ]:
#pip install ucimlrepo

# Packages
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 1: Prediction with Back-Propagation and Linear Regression

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez
Student: Natzaret Gálvez Rísquez

Part 1: Selecting and analyzing the datasets

We perform the predictions on  three datasets: 

In [ ]:
# We upload the datasets

# First dataset: File: A1-turbine.txt
    # 5 features: the first 4 are the input variables, the last one is the value to predict
    # 451 patterns: use the first 85% for training and validation, and the remaining 15% for test
df_turbine=pd.read_csv('C:/Users/Gari/Desktop/NEC/A1-turbine.txt', sep='\t', header=None)
header_vector_turbine = df_turbine.iloc[0, :].tolist() #header
df_turbine=df_turbine.iloc[1:,:]
df_turbine=pd.DataFrame(df_turbine)

# Second dataset: File: A1-synthetic.txt
    # 10 features: the first 9 are the input variables, the last one is the value to predict
    # 1000 patterns: use the first 80% for training and validation, and the remaining 20% for test
df_synthetic=pd.read_csv('C:/Users/Gari/Desktop/NEC/A1-synthetic.txt', sep='\t', header=None)
header_vector_synthetic = df_synthetic.iloc[0, :].tolist() #header
df_synthetic=df_synthetic.iloc[1:,:]
df_synthetic=pd.DataFrame(df_synthetic)

# Third dataset: from "https://archive.ics.uci.edu/dataset/186/wine+quality"
    # At least 6 features, one of them used for prediction
    # The prediction variable must take real (float or double) values; it should not represent a categorical value (that would correspond to a classification task)
    # At least 400 patterns
    # Select randomly 80% of the patterns for training and validation, and the remaining 20% for test; it is important to shuffle the original data, to destroy any kind of sorting it could have

# Wine Quality dataset [6496 rows x 11 columns]
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
df_wineQuality = wine_quality.data.features 
y = wine_quality.data.targets #quality of wine, an integer
  
# metadata 
#print(wine_quality.metadata) 
# variable information 
#print(wine_quality.variables) 

header_vector_wineQuality = df_wineQuality.columns.tolist() #header

In [ ]:
# As we can observe by the following header of the wine quality, alcohol level is the last feature
# We will use it as the value to predict
print(header_vector_wineQuality)

Now, we will do the data preprocessing to later do the data splitting.

In [ ]:
# Handling missing values, we check for and handle any missing values in our datasets
# Categorical values, if there are categorical variables, we encode them appropriately
# Outliers, we identify and handle the outliers in the data
# Normalization, in case is needed

# Data Preprocessing for Dataset 1 and 2
# - Normalize input and output variables
# - No need to preprocess (datasets already cleaned)

# Data Preprocessing for Dataset 3
# - Link to the source webpage to the documentation: "https://archive.ics.uci.edu/dataset/186/wine+quality"
# - Check for missing values, represent categorical values, look for outliers
# - Normalize input/output variables if needed

In [ ]:
##Turbine dataset
X_turbine = df_turbine.iloc[:, :-1]  # Features (all columns except the last one)
y_turbine = df_turbine.iloc[:, -1]   # Target variable (last column)

scaler_turbine = MinMaxScaler()
X_turbine_normalized = scaler_turbine.fit_transform(X_turbine)
#y_turbine_normalized = scaler_turbine.fit_transform(y_turbine.values.reshape(-1, 1))
# Because the prediction column has all NaN values, it is not necessary to reshape

In [ ]:
##Synthetic dataset
X_synthetic = df_synthetic.iloc[:, :-1]
y_synthetic = df_synthetic.iloc[:, -1]

# Normalize input and output variables
scaler_synthetic = MinMaxScaler()
X_synthetic_normalized = scaler_synthetic.fit_transform(X_synthetic)
y_synthetic_normalized = scaler_synthetic.fit_transform(y_synthetic.values.reshape(-1, 1))

In [ ]:
##Wine Quality dataset
#By the owners we know that this dataset has not missing values, we can check by:
missing_values_count = df_wineQuality.isnull().sum().sum()
print(f"Number of missing values in Wine Quality dataset: {missing_values_count}")

In [ ]:
##Wine Quality dataset
# No categorical variables in this dataset
# Identify and handle outliers using IQR method
def handle_outliers_iqr(data, threshold=1.5):
    data_copy = data.copy()  # Create a copy to avoid SettingWithCopyWarning
    Q1 = data_copy.quantile(0.25)
    Q3 = data_copy.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    data_copy[(data_copy < lower_bound) | (data_copy > upper_bound)] = np.nan
    return data_copy

# Handle outliers in all feature variables (columns) of df_wineQuality
df_wineQuality_no_outliers = handle_outliers_iqr(df_wineQuality)

#Shuffle
df_wineQuality_shuffled = df_wineQuality_no_outliers.sample(frac=1, random_state=42)

X_wineQuality = df_wineQuality_shuffled.iloc[:, :-1]
y_wineQuality = df_wineQuality_shuffled.iloc[:, -1]

# Normalize input and output variables
scaler_wineQuality = StandardScaler()
X_wineQuality_normalized_no_outliers = scaler_wineQuality.fit_transform(X_wineQuality)
y_wineQuality_normalized_no_outliers = scaler_wineQuality.fit_transform(y_wineQuality.values.reshape(-1, 1))

Now, we divide the datasets into validation & training and test.

In [ ]:
#First dataset, turbine
# Split the data into validation-training and testing sets
# Extract the first 85% for training
# Extract the remaining 15% for testing
# Splitting Turbine dataset
X_train_turbine, X_test_turbine, y_train_turbine, y_test_turbine = train_test_split(
    X_turbine_normalized, y_turbine, test_size=0.15, random_state=42
)

#Second dataset, synthetic
X_train_synthetic, X_test_synthetic, y_train_synthetic, y_test_synthetic = train_test_split(
    X_synthetic_normalized, y_synthetic_normalized, test_size=0.2, random_state=42
)

#Third dataset, wineQuality
X_train_wineQuality, X_test_wineQuality, y_train_wineQuality, y_test_wineQuality = train_test_split(
    X_wineQuality_normalized_no_outliers,
    y_wineQuality_normalized_no_outliers,
    test_size=0.2,
    random_state=42,
)

# Print the sizes of the datasets
#print("Total data size:", len(df_wineQuality))
#print("Training data size:", len(df_wineQualityTrainingValidation))
#print("Test data size:", len(df_wineQualityTesting))

Part 2: Implementation of BP

In [ ]:
class MyNeuralNetwork:
    def __init__(self, n_units, epochs, learning_rate, momentum, activation, validation_percentage):
        self.L = len(n_units) - 1
        self.n = n_units
        self.h = [None] * (self.L + 1)
        self.xi = [None] * (self.L + 1)
        self.w = [None] * (self.L + 1)
        self.theta = [None] * (self.L + 1)
        self.delta = [None] * (self.L + 1)
        self.d_w = [None] * (self.L + 1)
        self.d_theta = [None] * (self.L + 1)
        self.d_w_prev = [None] * (self.L + 1)
        self.d_theta_prev = [None] * (self.L + 1)
        self.fact = activation
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.validation_percentage = validation_percentage

    def fit(self, X, y):
        # Split the data into training and validation sets
        if self.validation_percentage > 0:
            num_validation = int(self.validation_percentage * X.shape[0])
            X_train, y_train = X[:-num_validation], y[:-num_validation]
            X_val, y_val = X[-num_validation:], y[-num_validation:]
        else:
            X_train, y_train = X, y

        # Initialize weights and thresholds
        for l in range(1, self.L + 1):
            self.w[l] = np.random.randn(self.n[l], self.n[l-1])
            self.theta[l] = np.random.randn(self.n[l])
            self.d_w_prev[l] = np.zeros_like(self.w[l])
            self.d_theta_prev[l] = np.zeros_like(self.theta[l])

        # Perform training for the specified number of epochs
        training_error = []
        validation_error = []
        for epoch in range(self.epochs):
            # Iterate over each training example
            for i in range(X_train.shape[0]):
                # Set the input layer activation
                self.xi[1] = X_train[i]

                # Feed-forward propagation
                for l in range(2, self.L + 1):
                    self.h[l] = np.dot(self.w[l], self.xi[l-1]) + self.theta[l]
                    self.xi[l] = self.activation_function(self.h[l])

                # Backward propagation
                self.delta[self.L] = self.sigmoid_derivative(self.xi[self.L]) * (self.xi[self.L] - y_train[i])
                for l in range(self.L - 1, 1, -1):
                    self.delta[l] = self.sigmoid_derivative(self.xi[l]) * np.dot(self.delta[l+1], self.w[l+1])

                # Update weights and thresholds
                for l in range(2, self.L + 1):
                    self.d_w[l] = -self.learning_rate * np.outer(self.delta[l], self.xi[l-1])
                    self.d_theta[l] = -self.learning_rate * self.delta[l]
                    self.w[l] += self.d_w[l] + self.momentum * self.d_w_prev[l]
                    self.theta[l] += self.d_theta[l] + self.momentum * self.d_theta_prev[l]
                    self.d_w_prev[l] = self.d_w[l]
                    self.d_theta_prev[l] = self.d_theta[l]

            # Calculate training error
            predictions_train = self.predict(X_train)
            error_train = np.mean((y_train - predictions_train) ** 2)
            training_error.append(error_train)

            # Calculate validation error if validation set is used
            if self.validation_percentage > 0:
                predictions_val = self.predict(X_val)
                error_val = np.mean((y_val - predictions_val) ** 2)
                validation_error.append(error_val)

        return training_error, validation_error
    
    def predict(self, X):
        predictions = []
        for i in range(X.shape[0]):
            self.xi[1] = X[i]
            for l in range(2, self.L + 1):
                self.h[l] = np.dot(self.w[l], self.xi[l-1]) + self.theta[l]
                self.xi[l] = self.activation_function(self.h[l])
            predictions.append(self.xi[self.L])
        return np.array(predictions)
    
    def loss_epochs(self):
        return np.array(self.training_errors), np.array(self.validation_errors) # 2 arrays of size (n_epochs, 2) that contain the evolution of 
                                                                                # the training error and the validation error for each of the epochs of the system

    def activation_function(self, x):
        if self.fact == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.fact == 'linear':
            return x
        elif self.fact == 'tanh':
            return np.tanh(x)
        elif self.fact == 'relu':
            return np.maximum(0, x)
        else:
            raise ValueError("Invalid activation function type.")

    def sigmoid_derivative(self, x):
        return x * (1 - x)
    

In [ ]:
# Create an instance of MyNeuralNetwork and train the model
model = MyNeuralNetwork(n_units=[4, 9, 1], epochs=1000, learning_rate=0.1, momentum=0.9, activation='sigmoid', validation_percentage=0.2)
training_error, validation_error = model.fit(X_train_synthetic, y_train_synthetic)

# Make predictions using the trained model
predictions = model.predict(X_test_synthetic)
print(predictions)

In [ ]:
training_errors, validation_errors = model.loss_epochs()
plt.plot(training_errors, label='Training Error')
plt.plot(validation_errors, label='Validation Error')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

Part 3: Obtaining and comparing predictions using the three models (BP, BP-F, MLR-F)

Part 3.1: Parameter comparison and selection

Part 3.2: Model result comparison